In [2]:
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
playersdatadf = pd.read_csv("player_data.csv")
#print(playersdatadf.head())
# player carrer pos height metrics
playersdf = pd.read_csv("Players.csv")
#print(playersdf.head())
# born / from stuff
stats_df = pd.read_csv("Seasons_stats.csv")

stats_df["ppg"] = stats_df["PTS"] / stats_df["G"]
stats_df["astpg"] = stats_df["AST"] / stats_df["G"]
stats_df["trbpg"] = stats_df["TRB"] / stats_df["G"]
# add columns for stats per game

# Creating a new statistic (Hybrid stat) to evaluate players more wholistically
ppg_weighting = 0.33333
astpg_weighting = 0.33333
trbpg_weighting = 0.33333

# Create the Hybrid Stat using a weighted avg
stats_df['Hybrid stat'] = (stats_df['ppg'] * ppg_weighting 
                                  + stats_df['astpg'] * astpg_weighting 
                                  + stats_df['trbpg'] * trbpg_weighting)

stats_df['Player'] = stats_df['Player'].str.replace('*', '')
#print(stats_df.loc[stats_df["ppg"].idxmax()])
#print(stats_df.loc[stats_df["astpg"].idxmax()])
#print(stats_df.loc[stats_df["trbpg"].idxmax()])


In [4]:

# highestppg_df = stats_df.loc[stats_df.groupby("Player")['ppg'].idxmax()]
# highestppg_df.info()

# top10 = highestppg_df.sort_values(by="ppg", ascending=False).head(10)

# chart = alt.Chart(top10).mark_bar().encode(
#     x=alt.X('Player', title='Player'),
#     y=alt.Y('ppg', title='Points Per Game (ppg)'),
#     color=alt.Color('WS', scale=alt.Scale(scheme='redblue'), title='WS')
# ).properties(
#     title="Top 10 PPG with Bar Shading Based on Win Shares",
#     width=500,
#     height=300
# )
# chart


In [5]:
# highestast_df = stats_df.loc[stats_df.groupby("Player")['astpg'].idxmax()]
# highestast_df.info()

# top10 = highestast_df.sort_values(by="astpg", ascending = False).head(10)

# chart = alt.Chart(top10).mark_bar().encode(
#     x=alt.X('Player', title='Player'),
#     y=alt.Y('astpg', title='Assists Per Game (astpg)'),
#     color=alt.Color('WS', scale=alt.Scale(scheme='redblue'), title='WS')
# ).properties(
#     title="Top 10 astpg with Bar Shading Based on Win Shares",
#     width=500,
#     height=300
# )
# chart


In [7]:
import altair as alt
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'stats_df' is already loaded
year_list = [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
recent_stats_df = stats_df[stats_df['Year'].isin(year_list)]

# Filter for positions you want to include
valid_positions = ['PG', 'PF', 'SG', 'C', 'SF']
recent_stats_df = recent_stats_df[recent_stats_df['Pos'].isin(valid_positions)]


# Creating a radio button selection for the Year (corrected)
year_selection = alt.selection_single(
    name = "year_select",
    fields = ["Year"],
    bind = alt.binding_radio(options = [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017], 
                           name = "Select Year:")
)

# Plotting the Hybrid Stat with PPG, ASTPG, and TRBPG to determine if there is a correlation
selection = alt.selection_interval(encodings=['x'])

# Create a legend selection for 'Tm' to allow interaction through the legend
Pos_legend_selection = alt.selection_point(
    fields = ['Pos'],  # Field to select from
    name = "Pos_legend_selection",  # Name of the selection
    bind = 'legend'
)

# First plot: Hybrid stat vs PPG
plot_one = alt.Chart(recent_stats_df).mark_circle(size = 100).encode(
    x = alt.X('Hybrid stat:Q', title = 'Hybrid Stat combining PPG, Astpg, and TRBpg'),
    y = alt.Y('ppg:Q', title = 'Points Scored per Game'),
    color = alt.condition(
        Pos_legend_selection,  # Apply color condition based on selection
        'Pos:N',  # Retain original color for selected positions
        alt.value('gray')  # Change color to gray for unselected teams
    ),
    opacity = alt.condition(
        Pos_legend_selection,  # Apply opacity condition based on selection
        alt.value(1),  # Full opacity for selected country
        alt.value(0.01)  # Light grey for unselected countries
    ),
    tooltip = ['Player:N', 'Age:Q', 'ppg:Q', 'Hybrid stat:Q']
).add_selection(
    selection, year_selection, Pos_legend_selection # Add both interval and year selection to the chart
).transform_filter(
    year_selection
  # Apply the year filter dynamically to the data
).properties(
    title='Relationship between Hybrid Stat and PPG'
)

# Second plot: Hybrid stat vs ASTPG
plot_two = alt.Chart(recent_stats_df).mark_circle(size=100).encode(
    x = alt.X('Hybrid stat:Q', title='Hybrid Stat combining PPG, Astpg, and TRBpg'),
    y = alt.Y('astpg:Q', title='Assists earned per Game'),
    color = alt.condition(
        Pos_legend_selection,  # Apply color condition based on selection
        'Pos:N',  # Retain original color for selected teams
        alt.value('gray')  # Change color to gray for unselected teams
    ),
    opacity = alt.condition(
        Pos_legend_selection,  # Apply opacity condition based on selection
        alt.value(1),  # Full opacity for selected country
        alt.value(0.05)  # Light grey for unselected countries
    ),
    tooltip = ['Player:N', 'Age:Q', 'ppg:Q', 'Hybrid stat:Q']
).add_selection(
    selection, year_selection, Pos_legend_selection  # Add both interval and year selection to the chart
).transform_filter(
    year_selection, selection  # Apply the year filter dynamically to the data
).properties(
    title = 'Relationship between Hybrid Stat and ASTPG'
)

final_chart = alt.hconcat(plot_one, plot_two)

# Display the final chart
final_chart.show()

# final_chart.save('Interactive_Scatter_Plot_Hybrid_Stat_vs_PPG_and_Hybrid_Stat_vs_Astpg_sorted_by_Position_and_Year.html')


/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/3194147798.py:15: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  year_selection = alt.selection_single(
/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/3194147798.py:47: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(
/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/3194147798.py:71: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

alt.HConcatChart(...)

In [8]:
import plotly.express as px

selected_cols = ['TS%', "3P%", "2P%", "FT%", "Year"]
mtx_df = stats_df[selected_cols]

# Data Preprocessing
mtx_df = mtx_df[mtx_df['Year'] > 1975]
mtx_df.dropna(inplace=True)
mtx_df.drop("Year", axis=1, inplace=True)

# Calculate the correlation matrix
matrix = mtx_df.corr()

matrix = matrix.round(3)

# Create the heatmap with Plotly (add text_auto to display correlation values)
fig = px.imshow(matrix,
                color_continuous_scale = 'viridis',  # You can change the color scale here
                zmin=-1,  # Set range for color scale (correlation values between -1 and 1)
                zmax=1,
                title="Correlation Matrix of NBA % Shooting Metrics",
                text_auto = True)  # This adds the correlation numbers inside the heatmap cells

# Display the plot
fig.show()

# Optionally, save the plot to an HTML file
# fig.write_html("Shooting_stats_corr_matrix_plotly.html")

In [9]:
selected_cols = ["Hybrid stat", "ppg","astpg", "Year"]
mtx_df = stats_df[selected_cols]
print(mtx_df['Year'].dtype)
mtx_df = mtx_df[mtx_df['Year'] > 1975]
mtx_df.dropna(inplace=True)
mtx_df.drop("Year",axis=1,inplace=True)
matrix = mtx_df.corr()

matrix = matrix.round(3)

# Create the heatmap with Plotly (add text_auto to display correlation values)
fig = px.imshow(matrix,
                color_continuous_scale = 'viridis',  # You can change the color scale here
                zmin = -1,  # Set range for color scale (correlation values between -1 and 1)
                zmax = 1,
                title = "Correlation Matrix of key NBA Stats",
                text_auto = True)  # This adds the correlation numbers inside the heatmap cells

# Display the plot
fig.show()

# Optionally, save the plot to an HTML file
# fig.write_html("Major_NBA_Stats_corr_matrix.html")

float64


In [10]:
# Select relevant columns and remove NaN values
stats_subset = stats_df[['Player', 'Year', 'ppg', 'OBPM']].dropna()

# Aggregate to get career averages
player_avg_stats = stats_subset.groupby("Player")[['ppg', 'OBPM']].mean().reset_index()

# Create Altair scatter plot
scatter_chart = alt.Chart(player_avg_stats).mark_circle(size=60).encode(
    x=alt.X('OBPM', title="Offensive Box Plus-Minus (OBPM)"),
    y=alt.Y('ppg', title="Points Per Game (PPG)"),
    tooltip=['Player', 'ppg', 'OBPM'],
    color=alt.Color('ppg', scale=alt.Scale(scheme='blues'), legend=alt.Legend(title="PPG"))
).properties(
    title="Interactive Scatter Plot: OBPM vs PPG",
    width=700,
    height=500
).interactive()

scatter_chart = alt.Chart(player_avg_stats).mark_circle(size=60).encode(
    x=alt.X('OBPM', title="Offensive Box Plus-Minus (OBPM)"),
    y=alt.Y('ppg', title="Points Per Game (PPG)"),
    tooltip=['Player', 'ppg', 'OBPM'],
    color=alt.Color('ppg', scale=alt.Scale(scheme='viridis'), legend=alt.Legend(title="PPG"))
).properties(
    title="Interactive Scatter Plot: OBPM vs PPG",
    width=700,
    height=500
).interactive()

scatter_chart

alt.Chart(...)

In [11]:
# Filter the data for the year 2017
recent_stats_2017 = recent_stats_df[recent_stats_df['Year'] == 2017]

# Group by 'Age' and calculate the mean of 'True Shooting %'
# Assuming the 'True Shooting %' column is named 'TS%', replace with actual column name if different
points_by_age_2017 = recent_stats_2017.groupby(['Age', 'Pos'])[['TS%']].mean().reset_index()

# Optionally, check the DataFrame


# Export the grouped data to a CSV file
# points_by_age_2017.to_csv('average_true_shooting_by_age_2017_v2.csv', index = False)

points_by_age_2017.head(20)


Age Pos       TS%
0   19.0   C  0.504250
1   19.0  PF  0.484500
2   19.0  SF  0.539000
3   19.0  SG  0.518000
4   20.0   C  0.562200
5   20.0  PF  0.500000
6   20.0  PG  0.486200
7   20.0  SF  0.458000
8   20.0  SG  0.505333
9   21.0   C  0.571800
10  21.0  PF  0.469000
11  21.0  PG  0.486000
12  21.0  SF  0.500667
13  21.0  SG  0.545500
14  22.0   C  0.548600
15  22.0  PF  0.468333
16  22.0  PG  0.463462
17  22.0  SF  0.516786
18  22.0  SG  0.566000
19  23.0   C  0.580000

In [12]:
# import matplotlib.pyplot as plt

# # Plotting the data
# plt.plot(points_by_age_2017['Age'], points_by_age_2017['TS%'], marker='o', color='b', linestyle='-', markersize=6)

# # Adding labels and title
# plt.title('Average True Shooting % by Age (2017)', fontsize=14)
# plt.xlabel('Age', fontsize=12)
# plt.ylabel('Average True Shooting %', fontsize=12)

# # Display the plot
# plt.show()

# # Going to remake this plot using D3 and the updated CSV File I created.



In [13]:
recent_stats_df.head()

Unnamed: 0    Year               Player Pos   Age   Tm     G    GS  \
16490       16490  2004.0  Shareef Abdur-Rahim  PF  27.0  TOT  85.0  56.0   
16491       16491  2004.0  Shareef Abdur-Rahim  PF  27.0  ATL  53.0  53.0   
16492       16492  2004.0  Shareef Abdur-Rahim  PF  27.0  POR  32.0   3.0   
16493       16493  2004.0          Malik Allen  PF  25.0  MIA  45.0   6.0   
16494       16494  2004.0            Ray Allen  SG  28.0  SEA  56.0  56.0   

           MP   PER  ...    AST   STL   BLK    TOV     PF     PTS        ppg  \
16490  2684.0  19.9  ...  174.0  68.0  37.0  184.0  222.0  1384.0  16.282353   
16491  1956.0  21.2  ...  127.0  44.0  19.0  131.0  147.0  1065.0  20.094340   
16492   728.0  16.5  ...   47.0  24.0  18.0   53.0   75.0   319.0   9.968750   
16493   616.0  10.5  ...   16.0  12.0  28.0   27.0   81.0   191.0   4.244444   
16494  2152.0  21.7  ...  268.0  71.0  11.0  156.0  132.0  1287.0  22.982143   

          astpg     trbpg  Hybrid stat  
16490  2.047059  7.517647     8.615600  
16491  2.396226  9.339623    10.609957  
16492  1.468750  4.500000     5.312447  
16493  0.355556  2.644444     2.414791  
16494  4.785714  5.107143    10.958224  

[5 rows x 57 columns]

In [25]:
import altair as alt
import pandas as pd

# List of available years for filtering
year_list = [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

# Create a selection for the year (radio buttons)
year_selection = alt.selection_single(
    name = "year_select",
    fields = ["Year"],
    bind = alt.binding_radio(options = year_list, name = "Select Year:")
)

# Updating the dataframe with all the average percentages by position by year for ALL TEAMS

# Filtering the data for the selected year and team
filtered_data = recent_stats_df

# Define the specific columns to calculate averages for
columns_to_average = ['TS%', '3P%', '2P%', 'FT%']

# Group data by Year and Position and calculate averages only for the specified columns
average_stats = (
    filtered_data
    .groupby(['Year', 'Pos'])[columns_to_average]  # Select only the columns to average
    .mean()  # Calculate the mean for these columns
    .reset_index()
)

# Add a Player name "All Teams" for these summary rows
average_stats['Tm'] = 'All Teams'

average_stats.head(10)

# Combine the summary rows back into the original dataset
final_dataset = pd.concat([recent_stats_df, average_stats], ignore_index=True)

# Sort the final dataset for better readability (optional)
final_dataset = final_dataset.sort_values(['Year', 'Pos'])

# Create a selection for the team (dropdown menu)
team_selection = alt.selection_single(
    name = "team_select",
    fields = ["Tm"],
    bind = alt.binding_select(options = sorted(final_dataset['Tm'].unique().tolist()), name = "Select Team:")
)

# Bar chart to show TS% by Position for the selected team and year
ts_chart = alt.Chart(final_dataset).mark_bar().encode(
    x = alt.X('Pos:N', title = 'Position'),
    y = alt.Y('mean(TS%):Q', title = 'True Shooting Percentage (TS%)'),
    color = 'Pos:N',
    tooltip = ['Pos:N', alt.Tooltip('mean(TS%):Q', format = '.2f')]
).add_selection(
    year_selection,
    team_selection  # Adding team selection to the chart
).transform_filter(
    year_selection  # Apply year filter dynamically
).transform_filter(
    team_selection
).properties(
    title="True Shooting Percentage by Player Position"
)

# Bar chart to show Three point % by Position for the selected team and year
three_pt_chart = alt.Chart(final_dataset).mark_bar().encode(
    x = alt.X('Pos:N', title = 'Position'),
    y = alt.Y('mean(3P%):Q', title = 'Three Point Shooting Percentage (3P%)'),
    color = 'Pos:N',
    tooltip = ['Pos:N', alt.Tooltip('mean(3P%):Q', format = '.2f')]
).add_selection(
    year_selection,
    team_selection  # Adding team selection to the chart
).transform_filter(
    year_selection  # Apply year filter dynamically
).transform_filter(
    team_selection
).properties(
    title = "3 Point Shooting Percentage % by Player Position"
)

# Bar chart to show TS% by Position for the selected team and year
two_pt_chart = alt.Chart(final_dataset).mark_bar().encode(
    x = alt.X('Pos:N', title = 'Position'),
    y = alt.Y('mean(2P%):Q', title = '2 Point Shooting Percentage (2P%)'),
    color = 'Pos:N',
    tooltip = ['Pos:N', alt.Tooltip('mean(2P%):Q', format = '.2f')]
).add_selection(
    year_selection,
    team_selection  # Adding team selection to the chart
).transform_filter(
    year_selection  # Apply year filter dynamically
).transform_filter(
    team_selection
).properties(
    title = "2 Point Shooting Percentage % by Player Position"
)

# Bar chart to show TS% by Position for the selected team and year
ft_chart = alt.Chart(final_dataset).mark_bar().encode(
    x = alt.X('Pos:N', title = 'Position'),
    y = alt.Y('mean(FT%):Q', title = 'Free Throw %'),
    color = 'Pos:N',
    tooltip = ['Pos:N', alt.Tooltip('mean(FT%):Q', format = '.2f')]
).add_selection(
    year_selection,
    team_selection  # Adding team selection to the chart
).transform_filter(
    year_selection  # Apply year filter dynamically
).transform_filter(
    team_selection
).properties(
    title = "Free Throw Shooting Percentage % by Player Position"
)

# Display the chart
# ts_chart | three_pt_chart | two_pt_chart | ft_chart

# Combine the charts in a 2x2 grid using vconcat and hconcat
chart_top = alt.hconcat(ts_chart, three_pt_chart)  # First row: TS% and 3P%
chart_bottom = alt.hconcat(two_pt_chart, ft_chart)  # Second row: 2P% and FT%

# Combine the top and bottom rows vertically
final_chart = alt.hconcat(chart_top, chart_bottom)

# Display the final chart
final_chart.show()

# Save as an HTML
# final_chart.save('Interactive_Bar_Plot_Shooting_percentage_by_Position_by_Team_by_Year.html')


/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/3537319470.py:8: AltairDeprecationWarning:


Deprecated since `altair=5.0.0`. Use selection_point instead.

/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/3537319470.py:42: AltairDeprecationWarning:


Deprecated since `altair=5.0.0`. Use selection_point instead.

/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/3537319470.py:54: AltairDeprecationWarning:


Deprecated since `altair=5.0.0`. Use add_params instead.

/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/3537319470.py:71: AltairDeprecationWarning:


Deprecated since `altair=5.0.0`. Use add_params instead.

/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/3537319470.py:88: AltairDeprecationWarning:


Deprecated since `altair=5.0.0`. Use add_params instead.

/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/3537319470.py:105: AltairDeprecationWarning:


Deprecated since `altair=5.0.0`. U

alt.HConcatChart(...)

In [15]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

x = recent_stats_df[['Hybrid stat']]  # Feature (input variable)
y = recent_stats_df['ppg']  # Target (output variable)

# Step 1: Split the data into training and testing sets (70-30 split)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.35, random_state = 100)

# Step 2: Create a Linear Regression model and fit it on the training data
model = LinearRegression()
model.fit(x_train, y_train)

# Step 3: Make predictions on the test data
y_pred = model.predict(x_test)

# Step 4: Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {round(mse,3)}')

# Optional: Print the R-squared value to understand the goodness of fit
r2 = model.score(x_test, y_test)
print(f'R-squared value: {round(r2,3)}')


Mean Squared Error (MSE): 1.725
R-squared value: 0.946


In [16]:
recent_stats_df.items()

<generator object DataFrame.items at 0x12c4e5e40>

In [17]:
import plotly.express as px

# Create a box plot for True Shooting Percentage by Player Position
fig_pts = px.box(recent_stats_df,
             x='Pos',       # Player type (position)
             y='PTS',       # True Shooting Percentage
             color='Pos',   # Color by Player type
             title='Box Plot of Points scored by Player Position')

# Show the plot
fig_pts.show()


# Create a box plot for True Shooting Percentage by Player Position
fig_ast = px.box(recent_stats_df,
             x = 'Pos',       # Player type (position)
             y = 'AST',       # True Shooting Percentage
             color = 'Pos',   # Color by Player type
             title = 'Box Plot of True Shooting % by Player Position',
             hover_data = ['Player', 'AST', 'Pos'])

# Show the plot
fig_pts.show()


In [18]:
# recent_stats_df.head()

# Step 1: Group by season and calculate the Min and Max VORP values for each season
recent_stats_df['VORP_min_season'] = recent_stats_df.groupby('Year')['VORP'].transform('min')
recent_stats_df['VORP_max_season'] = recent_stats_df.groupby('Year')['VORP'].transform('max')

# Step 2: Apply Min-Max scaling to get the VORP percentage
recent_stats_df['VORP_percentage'] = 100 * (recent_stats_df['VORP'] - recent_stats_df['VORP_min_season']) / (recent_stats_df['VORP_max_season'] - recent_stats_df['VORP_min_season'])

# Step 3: Drop the intermediate columns, if no longer needed
recent_stats_df.drop(columns=['VORP_min_season', 'VORP_max_season'], inplace=True)

# recent_stats_df.to_csv('recent_stats_df_2004_2017.csv', index = False)



In [27]:
# Looking at the % VORP for the top hybrid stat players

# I want to make a bar chart showing the VORP % for the players with the highest hybrid stat

# want to use Altair

import altair as alt
import pandas as pd

# Assuming final_dataset is your DataFrame containing the NBA player data with VORP and hybrid stats

# Sort the DataFrame by 'Year' and 'HybridStat' within each year, in descending order
recent_stats_df_sorted = recent_stats_df.sort_values(by = ['Year', 'Hybrid stat'], ascending = [True, False])

recent_stats_df_sorted = recent_stats_df_sorted.drop_duplicates(subset = ['Year', 'Player'], keep = 'first')

# For each year (2004 - 2017), keep only the top 10 players based on the HybridStat
top_10_players_df = recent_stats_df_sorted.groupby('Year').head(10)

top_10_players_df.tail(20)

# List of available years for filtering
year_list = [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

# Create a selection for the year (radio buttons)
year_selection = alt.selection_single(
    name = "year_select",
    fields = ["Year"],
    bind=alt.binding_radio(options=year_list, name = "Select Year:")
)

vorp_chart = alt.Chart(top_10_players_df).mark_bar().encode(
    x = alt.X('Player:N', title = 'Player'),
    y = alt.Y('Hybrid stat:Q', title='Hybrid stat Value'),
    color = alt.Color('VORP_percentage:Q',
                      title='VORP %',
                      scale=alt.Scale(scheme='redblue', range=['red', 'blue'])),
    tooltip = ['Player:N', alt.Tooltip('VORP_percentage:Q', format='.2f')]
).add_selection(
    year_selection
).transform_filter(
    year_selection
).properties(
    title="VORP Percentage for Top Hybrid Stat Players by Year"
)
vorp_chart

# vorp_chart.save('vorp_chart.html')

/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/2814629143.py:26: AltairDeprecationWarning:


Deprecated since `altair=5.0.0`. Use selection_point instead.

/var/folders/4q/x979ncbx5yj889hd87kg2vs00000gn/T/ipykernel_55196/2814629143.py:39: AltairDeprecationWarning:


Deprecated since `altair=5.0.0`. Use add_params instead.

